In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

Extraer las incidencias de un partido en base a su URL en la página historiedeboca.com.ar
(Por ahora simplemente crea una lista y no funciona con tandas de penales)

In [38]:
def get_incidences(url):
    incidences = []
    
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        incidence_table = soup.find_all('table', class_='datosPartido')
        
        if incidence_table:
            for table in incidence_table:
                rows = table.find_all('tr')
                for row in rows:
                    columns = row.find_all('td')
                    if len(columns) >= 2:
                        minute = columns[0].get_text().strip()
                        text = columns[1].get_text().strip()
                        if 'Gol' in text:
                            split_text = text.split(' de ')
                            if len(split_text) == 2:
                                incidence_type = split_text[0]
                                pattern = r'\(([^()]{0,4})\)'
                                match = re.search(pattern, split_text[1])
                                if match:
                                    goalscorer = split_text[1]
                                    goalscorer = re.sub(f' {pattern}', '', goalscorer)
                                    team = 'Rival'
                                else:
                                    goalscorer = split_text[1]
                                    team = 'Boca'
                                incidences.append([minute, goalscorer, incidence_type, team])
                                
                        elif 'le atajó un penal' in text:
                            pattern = r'^(.*?) le (.*)$'
                            coincidence = re.match(pattern, text)
                            goalkeeper = coincidence.group(1).strip()
                            gk_pattern = r'\(([^()]{0,4})\)'
                            match = re.search(gk_pattern, goalkeeper)
                            if match:
                                goalkeeper = re.sub(f' {gk_pattern}', '', goalkeeper)
                                team = 'Rival'
                            else: team = 'Boca'
                            incidence_type = coincidence.group(2).strip()
                            incidence_match = re.search(gk_pattern, incidence_type)
                            if incidence_match:
                                incidence_type = re.sub(f' {gk_pattern}', '', incidence_type)
                            incidences.append([minute, goalkeeper, incidence_type, team])
                            
                        elif 'expulsado' in text:
                            player = re.sub(' expulsado', '', text)
                            pattern = r'\(([^()]{0,4})\)'
                            match = re.search(pattern, player)
                            if match:
                                player = re.sub(f' {pattern}', '', player)
                                team = 'Rival'
                            else: team = 'Boca'
                            incidence_type = 'Expulsado'
                            incidences.append([minute, player, incidence_type, team])
                        
                        elif 'desvió un penal' in text:
                            player = re.sub(' desvió un penal', '', text)
                            pattern = r'\(([^()]{0,4})\)'
                            match = re.search(pattern, player)
                            if match:
                                player = re.sub(f' {pattern}', '', player)
                                team = 'Rival'
                            else: team = 'Boca'
                            incidence_type = 'Desvió un penal'
                            incidences.append([minute, player, incidence_type, team])
                        
        else: incidences = ['None', 'None', 'No hubo incidencencias en el partido', 'None']
        return incidences
    else: return f'Error {response.status_code}'

In [39]:
url = 'https://www.historiadeboca.com.ar/partido/gimnasia-lp-1-boca-2-campeonato-2015/6041.html'

incidences = get_incidences(url)
incidences

[["22'", 'Benítez', 'Expulsado', 'Rival'],
 ["47'", 'Mazzola', '1-0  Gol', 'Rival'],
 ["49'", 'J. Calleri', 'Expulsado', 'Boca'],
 ["57'", 'N. Lodeiro', '1-1  Gol', 'Boca'],
 ["66'", 'Coronel', 'Expulsado', 'Rival'],
 ["90'", 'A. Chávez', '1-2  Gol', 'Boca']]